#### Link of the Pycaret docs : https://pycaret.readthedocs.io/en/latest/api/time_series.html

In [1]:
#importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import numpy as np 
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import imageio
import os
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
import pmdarima as pm


In [2]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbsoja_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date  producao  produtividade     area
0  1977   19256.0           1632  11797.3
1  1978   14017.0           1276  10985.1
2  1979   16514.0           1461  11304.8
3  1980   19435.0           1665  11669.9
4  1981   21284.0           1752  12147.1
5  1982   21605.0           1692  12771.1
6  1983   19015.0           1631  11658.2
7  1984   21178.0           1735  12205.6
8  1985   21175.0           1773  11940.2
9  1986   20265.0           1549  13083.5


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           47 non-null     int64  
 1   producao       47 non-null     float64
 2   produtividade  47 non-null     int64  
 3   area           47 non-null     float64
dtypes: float64(2), int64(2)
memory usage: 1.6 KB


### Avaliando a produção de soja

In [4]:
df.drop(['produtividade','area'], axis = 1,inplace=True)
df.head(10)


,date,producao
0,1977,19256.0
1,1978,14017.0
2,1979,16514.0
3,1980,19435.0
4,1981,21284.0
5,1982,21605.0
6,1983,19015.0
7,1984,21178.0
8,1985,21175.0
9,1986,20265.0


In [5]:
df.set_index('date',inplace=True)

In [6]:
df=df.sort_index(axis=0)
print(df.head(10))

      producao
date          
1977   19256.0
1978   14017.0
1979   16514.0
1980   19435.0
1981   21284.0
1982   21605.0
1983   19015.0
1984   21178.0
1985   21175.0
1986   20265.0


fh: Optional[int or list or np.array], default = 1

The forecast horizon to be used for forecasting. Default is set to 1 i.e. forecast one point ahead. When integer is passed it means N continuous points in the future without any gap. If you want to forecast values with gaps, you must pass an array e.g. np.arange([13, 25]) will skip the first 12 future points and forecast from the 13th point till the 24th point ahead (note in numpy right value is inclusive and left is exclusive).

In [7]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=42, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,42
1,Target,producao
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(47, 1)"
5,Transformed data shape,"(47, 1)"
6,Transformed train set shape,"(46, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [8]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

## Checking Train-Test Split

In [9]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

## Comparing Models

The most basic assumption of time series forecasting is that past patterns continue into the future. 
Based on this assumption, in the short-term future forecast, since the environment in which the data is generated is similar to the present, 
the uncertainty is small and the forecast can be well-predicted. However, as we move towards long-term future forecasting, the possibility of the environment in which data is generated is likely to change, and unpredictable situations occur, and as errors continue to accumulate, uncertainty about the forecast data is bound to increase.

We compare the performance of various models and try to select the 4 models with the best performance.

In [10]:
top4 = exp.compare_models(n_select = 4)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,08:52:59
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
ets,ETS,1.6405,1.1662,10244.3726,10244.3726,0.1072,0.1022,0.0300
exp_smooth,Exponential Smoothing,1.6454,1.1698,10278.5309,10278.5309,0.1071,0.1027,0.0333
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,2.0805,1.4832,13134.4397,13134.4397,0.1235,0.1338,0.0967
theta,Theta Forecaster,2.1441,1.5267,13468.5276,13468.5276,0.1340,0.1355,0.0333
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,2.2507,1.6049,14221.0769,14221.0769,0.1324,0.1463,0.2133
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,2.2560,1.6087,14250.5524,14250.5524,0.1335,0.1461,0.0533
auto_arima,Auto ARIMA,2.3010,1.6368,14407.9862,14407.9862,0.1461,0.1452,0.1567
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,2.3309,1.6628,14745.3764,14745.3764,0.1364,0.1516,0.0800
naive,Naive Forecaster,2.3324,1.6622,14688.9667,14688.9667,0.1443,0.1495,4.0767
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,2.3719,1.6909,14959.5009,14959.5009,0.1441,0.1529,0.0500


## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data. 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

## Difference Plot Using Order

In [11]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":50})

### theta 

In [12]:
theta = exp.create_model('theta')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = theta, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 9},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(theta, fh = 9)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019.0000,1.0911,0.7754,6825.4373,6825.4373,0.0665,0.0688
1,2020.0000,2.3737,1.6817,14644.0852,14644.0852,0.1681,0.1551
2,2021.0000,2.9675,2.1230,18936.0601,18936.0601,0.1674,0.1827
Mean,nan,2.1441,1.5267,13468.5276,13468.5276,0.1340,0.1355
SD,nan,0.7831,0.5610,5013.5315,5013.5315,0.0477,0.0485


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2023,106767.3796
2024,107696.7827
2025,108626.1859
2026,109555.5891
2027,110484.9922
2028,111414.3954
2029,112343.7986
2030,113273.2017
2031,114202.6049


###  ADA

In [13]:
ada_cds_dt = exp.create_model('ada_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ada_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,08:53:44
Status,. . . . . . . . . . . . . . . . . .,Selecting Estimator
Estimator,. . . . . . . . . . . . . . . . . .,AdaBoost w/ Cond. Deseasonalize & Detrending


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019.0000,2.5945,1.8439,16229.9631,16229.9631,0.1582,0.1718
1,2020.0000,0.4619,0.3273,2849.6487,2849.6487,0.0327,0.0322
2,2021.0000,3.1850,2.2786,20323.7071,20323.7071,0.1796,0.1974
Mean,nan,2.0805,1.4832,13134.4397,13134.4397,0.1235,0.1338
SD,nan,1.1696,0.8364,7462.0095,7462.0095,0.0648,0.0726


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## exp_smooth

In [14]:
exp_smooth  = exp.create_model('exp_smooth')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## ets

In [15]:

ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ets, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


## ALL GRAPHS

In [16]:
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ada_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})              

## Avaliando a Produtividade

In [17]:
#função para tratar campo data                                                                                                                                                                      
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbsoja_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date  producao  produtividade     area
0  1977   19256.0           1632  11797.3
1  1978   14017.0           1276  10985.1
2  1979   16514.0           1461  11304.8
3  1980   19435.0           1665  11669.9
4  1981   21284.0           1752  12147.1
5  1982   21605.0           1692  12771.1
6  1983   19015.0           1631  11658.2
7  1984   21178.0           1735  12205.6
8  1985   21175.0           1773  11940.2
9  1986   20265.0           1549  13083.5


In [18]:
df.drop(['producao','area'], axis = 1,inplace=True)
df.head(10)

,date,produtividade
0,1977,1632
1,1978,1276
2,1979,1461
3,1980,1665
4,1981,1752
5,1982,1692
6,1983,1631
7,1984,1735
8,1985,1773
9,1986,1549


In [19]:
df.set_index('date',inplace=True)

In [20]:
df=df.sort_index(axis=0)
print(df.head(10))

      produtividade
date               
1977           1632
1978           1276
1979           1461
1980           1665
1981           1752
1982           1692
1983           1631
1984           1735
1985           1773
1986           1549


In [46]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=15, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,15
1,Target,produtividade
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(47, 1)"
5,Transformed data shape,"(47, 1)"
6,Transformed train set shape,"(46, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [47]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})


In [48]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})


In [49]:
top4 = exp.compare_models(n_select = 4)


## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data.
 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

In [50]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":50})


####  lightgbm_cds_dt

In [51]:
lightgbm_cds_dt = exp.create_model('lightgbm_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = lightgbm_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(lightgbm_cds_dt, fh = 9)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019.0000,1.0809,0.7973,363.9387,363.9387,0.0657,0.0680
1,2020.0000,2.8562,2.1048,951.3643,951.3643,0.2179,0.1965
2,2021.0000,0.3756,0.2753,132.2646,132.2646,0.0252,0.0256
Mean,nan,1.4376,1.0591,482.5225,482.5225,0.1029,0.0967
SD,nan,1.0436,0.7695,344.7488,344.7488,0.0829,0.0727


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2023,5577.8054
2024,5568.0084
2025,5664.4412
2026,5760.8739
2027,5857.3067
2028,5953.7394
2029,6050.1722
2030,6146.6049
2031,6243.0377


In [52]:
xgboost_cds_dt = exp.create_model('xgboost_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = xgboost_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(xgboost_cds_dt, fh = 9)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019.0000,0.3166,0.2335,106.5919,106.5919,0.0193,0.0191
1,2020.0000,4.0809,3.0074,1359.3283,1359.3283,0.3113,0.2694
2,2021.0000,1.1871,0.8701,417.9844,417.9844,0.0797,0.0830
Mean,nan,1.8615,1.3703,627.9682,627.9682,0.1368,0.1238
SD,nan,1.6091,1.1864,532.5455,532.5455,0.1259,0.1062


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2023,6102.3786
2024,5922.7912
2025,4754.7971
2026,5627.8459
2027,6488.1096
2028,6308.5221
2029,5140.5281
2030,6013.5769
2031,6873.8406


#### polytrend

In [53]:
polytrend = exp.create_model('polytrend')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = polytrend, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019.0000,1.2658,0.9336,426.1827,426.1827,0.0770,0.0801
1,2020.0000,2.6429,1.9476,880.3266,880.3266,0.2016,0.1831
2,2021.0000,0.0742,0.0544,26.1323,26.1323,0.0050,0.0050
Mean,nan,1.3276,0.9785,444.2139,444.2139,0.0945,0.0894
SD,nan,1.0496,0.7736,348.9564,348.9564,0.0812,0.0730


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


### THETa

In [54]:
theta = exp.create_model('theta')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(theta, fh = 9)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019.0000,0.0212,0.0156,7.1293,7.1293,0.0013,0.0013
1,2020.0000,3.7635,2.7735,1253.5985,1253.5985,0.2871,0.2510
2,2021.0000,0.7109,0.5211,250.3063,250.3063,0.0478,0.0489
Mean,nan,1.4985,1.1034,503.6780,503.6780,0.1120,0.1004
SD,nan,1.6262,1.1988,539.4869,539.4869,0.1252,0.1083


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2023,5210.7318
2024,5258.9481
2025,5307.1645
2026,5355.3809
2027,5403.5973
2028,5451.8136
2029,5500.0300
2030,5548.2464
2031,5596.4628


### ALL GRAPHS

In [55]:
exp.plot_model(estimator = lightgbm_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = xgboost_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = polytrend, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

In [ ]:
## Avaliando a Area

In [56]:
#função para tratar campo data                                                                                                                                                                      
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('dbsoja_br.xls', sheet_name='Planilha1')
print(df.head(10))

   date  producao  produtividade     area
0  1977   19256.0           1632  11797.3
1  1978   14017.0           1276  10985.1
2  1979   16514.0           1461  11304.8
3  1980   19435.0           1665  11669.9
4  1981   21284.0           1752  12147.1
5  1982   21605.0           1692  12771.1
6  1983   19015.0           1631  11658.2
7  1984   21178.0           1735  12205.6
8  1985   21175.0           1773  11940.2
9  1986   20265.0           1549  13083.5


In [57]:
df.drop(['producao','produtividade'], axis = 1,inplace=True)
df.set_index('date',inplace=True)
df=df.sort_index(axis=0)
print(df.head(10))

         area
date         
1977  11797.3
1978  10985.1
1979  11304.8
1980  11669.9
1981  12147.1
1982  12771.1
1983  11658.2
1984  12205.6
1985  11940.2
1986  13083.5


In [58]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=15, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,15
1,Target,area
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(47, 1)"
5,Transformed data shape,"(47, 1)"
6,Transformed train set shape,"(46, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [59]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

In [60]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})

In [61]:
top4 = exp.compare_models(n_select = 4)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
exp_smooth,Exponential Smoothing,0.8005,0.6865,929.1953,929.1953,0.0482,0.0499,0.0900
theta,Theta Forecaster,0.8628,0.7385,1007.9410,1007.9410,0.0517,0.0534,0.0267
ets,ETS,0.9606,0.8222,1123.6967,1123.6967,0.0575,0.0595,0.0433
auto_arima,Auto ARIMA,1.0488,0.8941,1241.4977,1241.4977,0.0623,0.0644,1.3333
naive,Naive Forecaster,1.1420,0.9707,1362.5667,1362.5667,0.0676,0.0700,0.0500
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,1.6599,1.4118,1977.3205,1977.3205,0.0983,0.1034,0.0533
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,1.6599,1.4118,1977.3199,1977.3199,0.0983,0.1034,0.0500
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,1.6599,1.4118,1977.3201,1977.3201,0.0983,0.1034,0.0700
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,1.6599,1.4118,1977.3200,1977.3200,0.0983,0.1034,0.0633
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,1.6599,1.4118,1977.3199,1977.3199,0.0983,0.1034,0.0700



## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data.
 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.


In [62]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":50})

In [63]:
exp_smooth = exp.create_model('exp_smooth')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(exp_smooth, fh = 9)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019.0000,1.5039,1.2986,1705.2784,1705.2784,0.0920,0.0965
1,2020.0000,0.5385,0.4617,634.1182,634.1182,0.0318,0.0323
2,2021.0000,0.3590,0.2992,448.1891,448.1891,0.0208,0.0210
Mean,nan,0.8005,0.6865,929.1953,929.1953,0.0482,0.0499
SD,nan,0.5028,0.4379,553.9983,553.9983,0.0313,0.0332


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2023,20549.4258
2024,20724.0199
2025,20449.3996
2026,21326.3979
2027,22586.1470
2028,21546.2980
2029,21719.7015
2030,21422.5363
2031,22331.7008


In [64]:
theta = exp.create_model('theta')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(theta, fh = 9)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019.0000,1.3906,1.2008,1576.8117,1576.8117,0.0851,0.0889
1,2020.0000,0.6838,0.5863,805.2358,805.2358,0.0404,0.0412
2,2021.0000,0.5140,0.4285,641.7756,641.7756,0.0297,0.0302
Mean,nan,0.8628,0.7385,1007.9410,1007.9410,0.0517,0.0534
SD,nan,0.3796,0.3332,407.7501,407.7501,0.0240,0.0255


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2023,20503.5134
2024,20777.3217
2025,20305.6457
2026,21006.7883
2027,21994.5313
2028,20831.9460
2029,21109.0774
2030,20628.8380
2031,21340.0792


In [65]:
ets= exp.create_model('ets')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(ets, fh = 9)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019.0000,1.3878,1.1984,1573.6449,1573.6449,0.0849,0.0887
1,2020.0000,0.9572,0.8207,1127.1231,1127.1231,0.0565,0.0582
2,2021.0000,0.5369,0.4476,670.3221,670.3221,0.0311,0.0316
Mean,nan,0.9606,0.8222,1123.6967,1123.6967,0.0575,0.0595
SD,nan,0.3474,0.3065,368.7880,368.7880,0.0220,0.0234


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2023,20413.4541
2024,20760.0601
2025,20461.8766
2026,21302.5478
2027,22536.7229
2028,21341.1744
2029,21695.0342
2030,21375.1947
2031,22244.9763


In [66]:
auto_arima= exp.create_model('auto_arima')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.plot_model(estimator = auto_arima, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":50})
exp.predict_model(auto_arima, fh = 9)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019.0000,1.0694,0.9234,1212.5506,1212.5506,0.0654,0.0677
1,2020.0000,1.1473,0.9837,1350.9898,1350.9898,0.0677,0.0701
2,2021.0000,0.9298,0.7751,1160.9527,1160.9527,0.0538,0.0553
Mean,nan,1.0488,0.8941,1241.4977,1241.4977,0.0623,0.0644
SD,nan,0.0900,0.0876,80.2370,80.2370,0.0061,0.0065


Decomposition is currently not supported for pandas dataframes without a PeriodIndex or DatetimeIndex. Please specify a PeriodIndex or DatetimeIndex in setup() before plotting decomposition plots.


,y_pred
2023,20413.4541
2024,20760.0601
2025,20461.8766
2026,21302.5478
2027,22536.7229
2028,21341.1744
2029,21695.0342
2030,21375.1947
2031,22244.9763
